Start!

In [1]:
!pip install timesfm[torch]
!pip install 'apache_beam[interactive,gcp,test] == 2.67.0'
!pip install google-generativeai

INFO: pip is looking at multiple versions of timesfm[torch] to determine which version is compatible with other requirements. This could take a while.


ERROR: Ignored the following yanked versions: 0.2.23, 0.3.18, 0.4.0
ERROR: Ignored the following versions that require a different python version: 0.0.11 Requires-Python <3.11,>=3.10; 0.0.12 Requires-Python <3.11,>=3.10; 0.1.1 Requires-Python <3.11,>=3.10; 0.1.14 Requires-Python <3.11,>=3.10; 0.1.15 Requires-Python <3.11,>=3.10; 0.1.16 Requires-Python <3.11,>=3.10; 0.1.17 Requires-Python <3.11,>=3.10; 0.1.18 Requires-Python <3.11,>=3.10; 0.1.2 Requires-Python <3.11,>=3.10; 0.1.21 Requires-Python <3.11,>=3.10; 0.1.22 Requires-Python <3.11,>=3.10; 0.1.23 Requires-Python <3.11,>=3.10; 0.1.24 Requires-Python <3.11,>=3.10; 0.1.25 Requires-Python <3.11,>=3.10; 0.1.3 Requires-Python <3.11,>=3.10; 0.1.31 Requires-Python <3.11,>=3.10; 0.1.32 Requires-Python <3.11,>=3.10; 0.1.4 Requires-Python <3.11,>=3.10; 0.1.5 Requires-Python <3.11,>=3.10; 0.1.6 Requires-Python <3.11,>=3.10; 0.1.7 Requires-Python <3.11,>=3.10; 0.4.14 Requires-Python >=3.9; 0.4.15 Requires-Python >=3.9; 0.4.16 Requires-Python 

  Using cached proto_plus-1.26.1-py3-none-any.whl.metadata (2.2 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.


In [3]:
import sys
print("PY:", sys.executable)
!{sys.executable} -m pip install -U pip
!{sys.executable} -m pip install apache-beam

PY: C:\Users\robotics\.conda\envs\tf_gpu\python.exe
  Using cached pip-25.0.1-py3-none-any.whl.metadata (3.7 kB)
Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.2
    Uninstalling pip-24.2:
      Successfully uninstalled pip-24.2
  Using cached apache_beam-2.60.0-cp38-cp38-win_amd64.whl.metadata (7.7 kB)
  Using cached crcmod-1.7.tar.gz (89 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached orjson-3.10.15-cp38-cp38-win_amd64.whl.metadata (42 kB)
  Using cached dill-0.3.1.1.tar.gz (151 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached cloudpickle-2.2.1-py3-none-any.whl.metadata (6.9 kB)
  Using cached fastavro-1.9.7-cp38-cp38-win_amd64.whl.metadata (5.6 kB)
  Using cached fasteners-0.20-py3-none-any.whl.metadata (4.8 kB)
  Using cached grpcio-1.65.5-cp38-cp38-win_amd64.whl.metada

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-intel 2.13.0 requires typing-extensions<4.6.0,>=3.6.6, but you have typing-extensions 4.12.2 which is incompatible.


In [3]:
import sys, subprocess, apache_beam as beam
print("Current Beam:", beam.__version__, "\nPython:", sys.executable)

# upgrade Beam in *this* kernel
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "apache-beam==2.69.0"])

Current Beam: 2.60.0 
Python: C:\Users\robotics\.conda\envs\tf_gpu\python.exe


CalledProcessError: Command '['C:\\Users\\robotics\\.conda\\envs\\tf_gpu\\python.exe', '-m', 'pip', 'install', '--upgrade', 'apache-beam==2.69.0']' returned non-zero exit status 1.

In [2]:
import logging

import apache_beam as beam
from apache_beam.coders import BooleanCoder
from apache_beam.coders import PickleCoder
from apache_beam.coders import TimestampCoder
from apache_beam.transforms.timeutil import TimeDomain
from apache_beam.transforms.userstate import OrderedListStateSpec
from apache_beam.transforms.userstate import ReadModifyWriteStateSpec
from apache_beam.transforms.userstate import TimerSpec
from apache_beam.transforms.userstate import on_timer
from apache_beam.utils.timestamp import MAX_TIMESTAMP
from apache_beam.utils.timestamp import Timestamp

_LOGGER = logging.getLogger(__name__)
logging.basicConfig(level=logging.INFO)
_LOGGER.setLevel(logging.INFO)


class OrderedSlidingWindowFn(beam.DoFn):

  ORDERED_BUFFER_STATE = OrderedListStateSpec('ordered_buffer', PickleCoder())
  WINDOW_TIMER = TimerSpec('window_timer', TimeDomain.WATERMARK)
  TIMER_STATE = ReadModifyWriteStateSpec('timer_state', BooleanCoder())
  EARLIEST_TS_STATE = ReadModifyWriteStateSpec('earliest_ts', TimestampCoder())

  def __init__(self, window_size, slide_interval):
    self.window_size = window_size
    self.slide_interval = slide_interval

  def start_bundle(self):
    _LOGGER.debug("start bundle")

  def finish_bundle(self):
    _LOGGER.debug("finish bundle")

  def process(
      self,
      element,
      timestamp=beam.DoFn.TimestampParam,
      ordered_buffer=beam.DoFn.StateParam(ORDERED_BUFFER_STATE),
      window_timer=beam.DoFn.TimerParam(WINDOW_TIMER),
      timer_state=beam.DoFn.StateParam(TIMER_STATE),
      earliest_ts_state=beam.DoFn.StateParam(EARLIEST_TS_STATE)):

    _, value = element
    ordered_buffer.add((timestamp, value))

    _LOGGER.debug("receive %s at %s", element, timestamp)
    timer_started = timer_state.read()

    earliest = earliest_ts_state.read()
    if not earliest or earliest > timestamp:
      earliest_ts_state.write(timestamp)

    if not timer_started:
      earliest_ts_state.write(timestamp)

      first_slide_start = int(
          timestamp.micros / 1e6 // self.slide_interval) * self.slide_interval
      first_slide_start_ts = Timestamp.of(first_slide_start)

      first_window_end_ts = first_slide_start_ts + self.window_size
      _LOGGER.debug("set timer to %s", first_window_end_ts)
      window_timer.set(first_window_end_ts)

      timer_state.write(True)

    return []

  @on_timer(WINDOW_TIMER)
  def on_timer(
      self,
      key=beam.DoFn.KeyParam,
      fire_ts=beam.DoFn.TimestampParam,
      ordered_buffer=beam.DoFn.StateParam(ORDERED_BUFFER_STATE),
      window_timer=beam.DoFn.TimerParam(WINDOW_TIMER),
      timer_state=beam.DoFn.StateParam(TIMER_STATE),
      earliest_ts_state=beam.DoFn.StateParam(EARLIEST_TS_STATE)):
    _LOGGER.debug("timer fire at %s", fire_ts)
    window_end_ts = fire_ts
    window_start_ts = window_end_ts - self.window_size

    window_values = list(
        ordered_buffer.read_range(window_start_ts, window_end_ts))

    _LOGGER.debug(
        "window start: %s, window end: %s", window_start_ts, window_end_ts)
    _LOGGER.debug("windowed data in buffer %s", str(window_values))
    if window_values:
      yield (key, (window_start_ts, window_end_ts, window_values))

    next_window_end_ts = fire_ts + self.slide_interval
    next_window_start_ts = window_start_ts + self.slide_interval

    earliest_ts = earliest_ts_state.read()
    ordered_buffer.clear_range(earliest_ts, next_window_start_ts)

    remaining_data = list(
        ordered_buffer.read_range(next_window_start_ts, MAX_TIMESTAMP))

    if not remaining_data:
      timer_state.clear()
      earliest_ts_state.write(next_window_start_ts)
      return

    _LOGGER.debug("set timer to %s", next_window_end_ts)
    window_timer.set(next_window_end_ts)


class FillGapsFn(beam.DoFn):
  def __init__(self, expected_interval: float):
    """
    Args:
      expected_interval: The expected time delta between elements, in seconds.
    """
    self.expected_interval = expected_interval

  def process(self, element):
    key, (window_start_ts, window_end_ts, window_elements) = element

    received_data = {
        round(float(ts.micros / 1e6), 5): val
        for ts, val in window_elements
    }

    start_sec = float(window_start_ts.micros / 1e6)
    end_sec = float(window_end_ts.micros / 1e6)

    filled_values = []
    current_ts_sec = start_sec

    while current_ts_sec < end_sec:
      lookup_ts = round(current_ts_sec, 5)

      if lookup_ts in received_data:
        filled_values.append(float(received_data[lookup_ts]))
      else:
        filled_values.append('NaN')

      current_ts_sec += self.expected_interval

    yield (key, (window_start_ts, window_end_ts, filled_values))

ImportError: cannot import name 'OrderedListStateSpec' from 'apache_beam.transforms.userstate' (C:\Users\robotics\.conda\envs\tf_gpu\lib\site-packages\apache_beam\transforms\userstate.py)